# Main File To Run Detections

In [ ]:
# on Colab make sure GPU is set up
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
    print('and then re-execute this cell.')
else:
    print(gpu_info)

Tue Dec  8 14:49:24 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    24W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# hook up your google drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
# set up an alias to access your drive
!ln -s /content/gdrive/My\ Drive/ /mydrive

ln: failed to create symbolic link '/mydrive/My Drive': File exists


In [ ]:
# enter this repo on the remote machine
cd /mydrive/nba-automation

/content/gdrive/My Drive/TensorFlow-2.x-YOLOv3


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from yolov3.utils import Load_Yolo_model, Load_Custom_Yolo_model, image_preprocess, postprocess_boxes, nms, draw_bbox, read_class_names, detect_video
from yolov3.configs import *
from yolov3.yolov3 import Create_Yolov3
import time

from object_tracker import Object_tracking
from slice_video import create_time_stamp_windows, video_slicer

from deep_sort import nn_matching
from deep_sort.detection import Detection
from deep_sort.tracker import Tracker
from deep_sort import generate_detections as gdet

In [ ]:
# load in the custom basketball model
yolo_custom = Load_Custom_Yolo_model()

GPUs [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# load in the darknet model so we can detect people
yolo = Load_Yolo_model()

GPUs [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# set up video path to grab and save to
video_path = "/mydrive/basketball-videos/made-basket-tracker/player-tracker.mp4"
save_path = "/mydrive/basketball-videos/made-basket-tracker/player-tracker-tracked.mp4"


In [ ]:
# Run the video info function
video_info = Object_tracking(yolo, video_path, save_path, input_size=YOLO_INPUT_SIZE, show=False, 
                              iou_threshold=0.1, rectangle_colors=(255,0,0), Track_only = ["person"], custom_yolo=yolo_custom)

Time: 7119.30ms, Detection FPS: 0.1, total FPS: 0.1
Time: 3613.37ms, Detection FPS: 0.3, total FPS: 0.3
Time: 2446.07ms, Detection FPS: 0.4, total FPS: 0.4
Time: 1862.12ms, Detection FPS: 0.5, total FPS: 0.5
Time: 1511.51ms, Detection FPS: 0.7, total FPS: 0.6
Time: 1277.79ms, Detection FPS: 0.8, total FPS: 0.7
Time: 1112.20ms, Detection FPS: 0.9, total FPS: 0.8
Time: 987.04ms, Detection FPS: 1.0, total FPS: 0.9
Time: 892.29ms, Detection FPS: 1.1, total FPS: 1.0
Time: 814.00ms, Detection FPS: 1.2, total FPS: 1.1
Time: 750.41ms, Detection FPS: 1.3, total FPS: 1.2
Time: 697.58ms, Detection FPS: 1.4, total FPS: 1.3
Time: 653.21ms, Detection FPS: 1.5, total FPS: 1.4
Time: 614.67ms, Detection FPS: 1.6, total FPS: 1.5
Time: 581.88ms, Detection FPS: 1.7, total FPS: 1.6
Time: 552.41ms, Detection FPS: 1.8, total FPS: 1.6
Time: 526.83ms, Detection FPS: 1.9, total FPS: 1.7
Time: 504.05ms, Detection FPS: 2.0, total FPS: 1.8
Time: 483.52ms, Detection FPS: 2.1, total FPS: 1.9
Time: 465.26ms, Detectio

In [ ]:
# this is just a safety stop if we did run all
assert False

AssertionError: ignored

In [ ]:
# Grab the dict that keeps te amount of baskets made by each team
baskets_dict = video_info["baskets_dict"]
baskets_dict

In [ ]:
# Grab the list of frames where a basket was scored
basket_frame_list = video_info["basket_frame_list"]
basket_frame_list

In [ ]:
# use the previous frames to create "frame windows" to feed into our video clipper
time_stamp_windows = create_time_stamp_windows(basket_frame_list, 5, 2, 30)
time_stamp_windows

In [ ]:
video_path = "/mydrive/basketball-videos/made-basket-tracker/game_3_clip_2_tracked_4.mp4"
save_path = "/mydrive/basketball-videos/made-basket-tracker/game_3_clip_2_tracked_4_highlights.mp4"

In [ ]:
# slice up our video
video_slicer(video_path, save_path, time_stamp_windows["start_frames"], time_stamp_windows["end_frames"])